<a href="https://colab.research.google.com/github/Alishba01234/PAI_PROJECT/blob/main/PAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving pesticidesData.xls to pesticidesData.xls


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pesticides_df = pd.read_excel('pesticidesData.xls', usecols=range(16))
df = pesticides_df.copy()
df.head()

,S.No.,R.No.,Date,Job No.,Firm Name,Brand Name,Common Name,Formulation,Quantity (lit/kg),C & F Value,Ex.Rate (US$),Total Value (Rs.),Origin,Reg.No.,PSI,Category
0,1.0,2.0,"02-January,2017",C/PK/QC16015,Exin Chemicals,Xiaowang,Mesotrione + Atrazine,55% SC,16000.0,3.30,106,5596800.0,China,R-1912,NMCI,Herb.
1,2.0,2.0,"02-January,2017",C/S/PK/170059/16,New Agri Care,Deft,Metsulfuron Methyl,10% WP,2000.0,3.37,106,714440.0,China,R-908,NMCI,Herb.
2,3.0,1.0,"02-January,2017",CU16119148,Tara Imperial Industries,NaN,Glyphosate,41% SL,32000.0,1.78,106,6037760.0,China,R-6945,CU,Herb.
3,4.0,4.0,"02-January,2017",CU16119110,Tara Imperial Industries,NaN,Metalaxyl + Mancozeb,72% WP,15000.0,3.60,106,5724000.0,China,R-4982,CU,Fungicide
4,5.0,4.0,"03-January,2017",PK-130617/16,Sun Crop Pesticides,Truce Xtra,Mesotrione + Atrazine,88.8% WG,1000.0,8.50,106,901000.0,China,R-1983,NMCI,Herb.


In [3]:
df['Total Value (Rs.)'] = df['Quantity (lit/kg)']*df['C & F Value']*df['Ex.Rate (US$)']

In [4]:
df.drop(columns=['S.No.','R.No.','Ex.Rate (US$)'],inplace=True)
df['Brand Name'].fillna("Form-16",inplace=True)

/tmp/ipython-input-340924966.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Brand Name'].fillna("Form-16",inplace=True)


In [5]:
from dateutil import parser

def parse_messy_date(date_str):
    try:
        parsed_date = parser.parse(date_str, dayfirst=True, fuzzy=True)
        return parsed_date.strftime('%Y-%m-%d')
    except (ValueError, TypeError):
        return np.nan

df['Date'] = df['Date'].apply(parse_messy_date)

In [6]:
df['Formulation'].value_counts()

,count
Formulation,
80% WG,133
25% WP,129
15% WP,67
72% WP,64
70% WP,52
...,...
25%SC,1
78% WP,1
WS 70,1


In [7]:
import re

# Regex to match either:
# 1. number (with optional %) at the start, then type
# 2. OR type at the start, number at the end
pattern = r"""
    ^\s*
    (?:
        (?P<Percentage1>\d+(?:\.\d+)?)       # number at start
        (?:%)?                               # optional %
        \s*(?P<Type1>.+)                     # type after number
    |
        (?P<Type2>[A-Za-z\s]+)               # type at start
        \s*(?P<Percentage2>\d+(?:\.\d+)?)    # number at end
    )
    \s*$
"""

# Use re.VERBOSE here
extracted = df['Formulation'].str.extract(pattern, flags=re.VERBOSE)

# Combine matched groups into final columns
df['Percentage'] = extracted['Percentage1'].combine_first(extracted['Percentage2'])
df['Type'] = extracted['Type1'].combine_first(extracted['Type2'])

# Clean up
df['Percentage'] = pd.to_numeric(df['Percentage'], errors='coerce')
df['Type'] = df['Type'].str.strip()

df.dropna(subset=['Percentage','Type'],inplace=True)

In [8]:
df = df.drop('Formulation', axis=1)

In [10]:
df['Type'] = df['Type'].str.upper()

In [11]:
df = df.rename(columns={
    'Common Name': 'Composition'
})

In [12]:
df['Category'].value_counts()

,count
Category,
Herb.,500
Insect.,429
Fung.,274
Fungicide,202
Insect,79
Fumigant,31
Biological,21
Fum.,3
Insecticide,2


In [13]:
category_mapping = {
    'Herb.': 'Herbicide',
    'Fung.': 'Fungicide',
    'Insect.': 'Insecticide',
    'Insect': 'Insecticide',
    'Fumigation': 'Fumigant',
    'Fum.': 'Fumigant',
    'Acari': 'Acaricide',
    'Acaracide': 'Acaricide',
    'Acracide': 'Acaricide',
}

In [14]:
# Normalize the Category column
df['Category'] = df['Category'].replace(category_mapping)

In [15]:
df['Category'].value_counts()

,count
Category,
Insecticide,510
Herbicide,500
Fungicide,476
Fumigant,35
Biological,21
Acaricide,3


In [27]:
import re

def normalize_firm_name(name):
    # Uppercase
    name = name.upper()

    # Remove punctuation
    name = re.sub(r'[^\w\s]', '', name)

    # Remove multiple spaces
    name = re.sub(r'\s+', ' ', name).strip()

    # Fix common keywords
    name = name.replace('PVT LTD', '')
    name = name.replace('PRIVATE LIMITED', '')
    name = name.replace('LIMITED', '')
    name = name.replace('LTD', '')
    name = name.replace('ENTERPRISES', 'ENTERPRISE')
    name = name.replace('SCIENCES', 'SCIENCE')
    name = name.replace('AGROSCIENCES', 'AGRO SCIENCE')

    return name.strip()

df['Firm Name Normalized'] = df['Firm Name'].apply(normalize_firm_name)

In [38]:
manual_firm_map = {
    # JAFFER AGRO
    'JAFFER AGRO SERVICES': 'JAFFER AGRO',
    'JAFFER AGRO SERVICE': 'JAFFER AGRO',
    'JAFFER AGRO SCIENCE': 'JAFFER AGRO',

    # SYNGENTA
    'SYNGENTA PAKISTAN': 'SYNGENTA',

    # FOUR BROTHERS
    'FOUR BROTHERS AGRI SERVICE': 'FOUR BROTHERS',
    'FOUR BROTHER BIOLOGIC': 'FOUR BROTHERS',
    'FOUR BROTHERS AGRI SERVICES': 'FOUR BROTHERS',
    'FOUR BROTHER AGRI SERVICES': 'FOUR BROTHERS',

    # SUN CROP
    'SUN CROP PESTICIDE': 'SUN CROP PESTICIDES',

    # NAFEES
    'NAFEES ENTERPRENUER': 'NAFEES',
    'NAFEES ENTREPRENEUR': 'NAFEES',
    'NAFEES ENTERPRISE': 'NAFEES',

    # HEXTAR
    'HEXTAR CHEMCIALS': 'HEXTAR CHEMICALS',
    'HEXTAR CHEMICAL': 'HEXTAR CHEMICALS',

    # WELCON / WELCONE
    'WELCONE CHEMICALS': 'WELCON CHEMICALS',
    'WELCON AGRO CHEMICALS': 'WELCON CHEMICALS',

    # RB AVARI
    'R B AVARI ENTERPRISES': 'RB AVARI ENTERPRISES',

    # WARBLE
    'WARBLE PVTLTD': 'WARBLE',

    # AGRIMORE
    'AGRIMORE PVT': 'AGRIMORE',

    # ORANGE PROTECTION
    'ORANGE PROTECTION PVT LTD': 'ORANGE PROTECTION',

    # AGROTREE
    'AGROTREE LIFE SCIENCE': 'AGROTREE LIFESCIENCE',

    # COMMANDER AGRO
    'COMMANDER AGRO CHEMICAL': 'COMMANDER AGRO',

    # ARYSTA
    'ARYSTA LIFE SCIENE': 'ARYSTA',
    'ARYSTA LIFESCIENE': 'ARYSTA',
    'ARYSTA LIFESCIENCE PAKISTAN': 'ARYSTA',

    # AGGRIO
    'AGGRIO PAKISTAN PVT LTD': 'AGGRIO',

    # DUPONT
    'DU PONT PAKISTAN OPERATIONS': 'DUPONT',

    # GREEN GRO
    'GREEN GRO PVT LTD': 'GREEN GRO',

    # ICI
    'ICI PAKISTAN LTD': 'ICI',

    'SUN CROP PESTICIDE': 'SUN CROP PESTICIDES',
    'FM UNITED': 'FMC UNITED',
    'NAFEES ENTERPRENEUR': 'NAFEES',
    'ARYSTA LIFE SCIENCE': 'ARYSTA',
    'ARYSTA LIFESCIENCE': 'ARYSTA',
    'HEXON CHEMICAL': 'HEXON CHEMICALS',
    'ALNOOR AGRO CHEMICALS': 'AL NOOR AGRO CHEMICALS',
    'AGGRIO PAKISTAN': 'AGGRIO',
    'AGGRIO PAKISTAN CHEMICALS': 'AGGRIO',
    'FOUR BROTHERS BIOLOGIC': 'FOUR BROTHERS',
    'FOUR BROTHERS BIOLOGIC AG': 'FOUR BROTHERS',
    'HELB AGRO SCIENCE': 'HELB',
    'HELB AGROSCIENCE': 'HELB',
    'HELB AGRO SCIENCES': 'HELB',
    'HELB PESTICIDES CHEMICALS': 'HELB',
    'HELB PESTISIDE': 'HELB',
    'AGRO': 'AGROW',
    'WARBLE PVT': 'WARBLE',
    'PEST MANAGEMENT': 'PEST MANAGEMENT SERVICES',
    'RICE CO PAKISTAN': 'RICECO PAKISTAN',
    'MS AGRICA CHEMICALS': 'AGRICA CHEMICALS',
    'CROPLANDS CHEMICALES': 'CROPLANDS CHEMICALS',
    'SESSA FARM CHEMICAL': 'SESSA FARM CHEMICALS',
    'S ESSA FARM CHEMICALS': 'SESSA FARM CHEMICALS',
    'S ESSA FARM CHEMICAL': 'SESSA FARM CHEMICALS',
    'DUPENT PAKISTAN OPERATION': 'DUPONT',
    'DUPONT PAKISTAN': 'DUPONT',
    'RAS INTERNATIONAL CORPORATION': 'RAS INTERNATIONAL',
}

df['Firm Name Standardized'] = df['Firm Name Normalized'].replace(manual_firm_map)

In [39]:
df['Firm Name Standardized'].value_counts()

,count
Firm Name Standardized,
JAFFER AGRO,73
BAYER PAKISTAN,66
SUN CROP PESTICIDES,60
PATRON CHEMICALS,56
AQ ENTERPRISE,55
WARBLE,53
SWAT AGRO CHEMICALS,51
SYNGENTA,47
LEADER AG,44


In [21]:
pd.set_option('display.max_rows', None)
df['Firm Name'].value_counts()

,count
Firm Name,
JAFFER AGRO SERVICES,66
BAYER PAKISTAN,64
SUN CROP PESTICIDES,58
PATRON CHEMICALS,55
SWAT AGRO CHEMICALS,44
AQ ENTERPRISES,42
SYNGENTA PAKISTAN,40
PLUS ENTERPRISES,39
FMC UNITED,38


In [ ]:
df.head()

In [ ]:
df.info()